In [1]:
import numpy as np
import matplotlib as plt
import random
import cirq

#### N = 4 elements, log2(N) qubits & 1 solution

In [117]:
def grover(qubits, fsol, reps):
    '''Grover-circuit simulator for 1 solution in N elements.
    Input: log_2(N) qubits + ancilla & function indicating right element.'''
    N = len(fsol)
    n = len(qubits) - 1
    
    # Create circuit
    circuit = cirq.Circuit()
    
    # Initialize qubits
    for qb in qubits[:-1]:
        circuit.append(cirq.H(qb))
    circuit.append([cirq.X(qubits[-1]), cirq.H(qubits[-1])])
        
    # Apply Grover operator int(pi/4 * sqrt(N)) times
    #Oracle
    for i in range(N):
        if fsol[i] == 1:
            break
    # convert position of fsol solution to binary, parse each bit & apply X gate if bit==0 
    oracleX = []
    for j,bit in enumerate(format(i, '0'+str(n)+'b')):
        if bit == '0':
            oracleX += [cirq.X(qubits[n-j-1])]
    oracle = oracleX + [cirq.CCX(*qubits)] + oracleX # only works for n=2
    
    for _ in range(int(np.pi/4 * np.sqrt(N))):
        #Oracle
        circuit.append(oracle, strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

        for qb in qubits[:-1]:
            circuit.append(cirq.H(qb))

        #Phase-shift
        for qb in qubits[:-1]:
            circuit.append(cirq.X(qb))
        circuit.append(cirq.CZ(*qubits[:-1])) # only works for n=2
        for qb in qubits[:-1]:
            circuit.append(cirq.X(qb))

        for qb in qubits[:-1]:
            circuit.append(cirq.H(qb))
    # print('Grover iterations:', _+1, '\n')
    
    # Measure
    circuit.append(cirq.measure(*qubits[:-1]))
    print(circuit, '\n')

    # Simulate circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=reps)
    print('Measurement result:', result)
    return circuit, result

In [131]:
N = 4
f = np.zeros(N, dtype=int)
sol = np.random.randint(N)
f[sol] = 1

qubits = cirq.LineQubit.range(int(np.log2(N)) + 1)
circuit, result = grover(qubits, f, 10)
print('\nSolution was element {}.'.format(sol))

0: ───H───────@───H───X───@───X───H───M───
              │           │           │
1: ───H───────@───H───X───@───X───H───M───
              │
2: ───X───H───X─────────────────────────── 

Measurement result: 0,1=1111111111, 1111111111

Solution was element 3.
